In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import re
import pandas as pd

IN_PATH = "/kaggle/input/data-analyst-job-roles-in-canada/Cleaned_Dataset.csv"
df = pd.read_csv(IN_PATH)

# 1) 모든 컬럼을 snake_case로 통일
def snake(s): 
    return re.sub(r'[^a-z0-9]+', '_', s.strip().lower()).strip('_')
original_cols = list(df.columns)
df.columns = [snake(c) for c in df.columns]

print("Original columns:", original_cols)
print("Normalized columns:", list(df.columns))

# 2) 동의어 매핑(데이터셋마다 다를 수 있는 이름들을 통일)
rename_map = {}

# 직무/회사/지역
for a,b in [
    ("company_name","employer"), 
    ("jobrole","job_title"), ("title","job_title"), 
    ("city_name","city"),
    ("state","province"), ("prov","province")
]:
    if a in df.columns and b not in df.columns:
        rename_map[a] = b

# 급여: 다양한 표기 대응
salary_name_candidates = {
    "min_salary": ["min_salary","minimum_salary","salary_min","min","minsal"],
    "max_salary": ["max_salary","maximum_salary","salary_max","max","maxsal"],
    "avg_salary": ["avg_salary","average_salary","salary_avg","mean_salary","median_salary","mid_salary","mid"]
}
for target, cands in salary_name_candidates.items():
    if target not in df.columns:
        for c in cands:
            if c in df.columns:
                rename_map[c] = target
                break

# 스킬: 혹시 'skill' 컬럼이 다른 이름일 때
if "skill" not in df.columns:
    for c in ["skills","language_and_tools","tools","required_skills"]:
        if c in df.columns:
            rename_map[c] = "skill"
            break

if rename_map:
    df = df.rename(columns=rename_map)
    print("Applied renames:", rename_map)

# 3) Power BI용 컬럼만 추리기 (존재하는 것만)
keep = [
    "job_title", "employer", "city", "province",
    "min_salary", "max_salary", "avg_salary",
    "sql", "python", "excel", "tableau", "power_bi"
]
keep = [c for c in keep if c in df.columns]
print("Keeping columns:", keep)

# 방어: 그래도 keep이 0개면 무엇이 있는지 보여주고 종료
if not keep:
    print("⚠️ No matching columns found for BI export. Available columns are:")
    print(list(df.columns))
else:
    bi = df[keep].copy()
    print("BI shape:", bi.shape)
    display(bi.head(3))
    bi.to_csv("clean_for_bi.csv", index=False)
    print("✅ Saved: clean_for_bi.csv")


Original columns: ['Job Title', 'Job Info', 'Position', 'Employer', 'City', 'Province', 'Skill', 'Seniority', 'Work Type', 'Industry Type', 'Min_Salary', 'Max_Salary', 'Avg_Salary']
Normalized columns: ['job_title', 'job_info', 'position', 'employer', 'city', 'province', 'skill', 'seniority', 'work_type', 'industry_type', 'min_salary', 'max_salary', 'avg_salary']
Keeping columns: ['job_title', 'employer', 'city', 'province', 'min_salary', 'max_salary', 'avg_salary']
BI shape: (1796, 7)


,job_title,employer,city,province,min_salary,max_salary,avg_salary
0,Systems and Data Analysts,Binance,Remote,Undef,76000.00,89440.0,82720.000
1,Business and Marketing Analysts,Canadian Nuclear Laboratories,Remote,Undef,65000.00,87000.0,76000.000
2,Business and Technical Analysts,Sander Geophysics Limited,Ottawa,ON,64123.59,86600.0,75361.795


✅ Saved: clean_for_bi.csv
